# RAG Based on parent document
- Hybrid Search
- ReRanker
- [Parent Document](https://medium.aiplanet.com/advanced-rag-providing-broader-context-to-llms-using-parentdocumentretriever-cc627762305a)
    

## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. Titan Embedding 및 LLM 인 Claude-v3.5 모델 로딩

### LLM 로딩 (Claude-v3.5)

In [5]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    #streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f79ac932c80>], client=<botocore.client.BedrockRuntime object at 0x7f7990fa88b0>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']})

### Embedding 모델 선택

In [43]:
from langchain_aws import BedrockEmbeddings

In [44]:
llm_emb = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-Embeddings-V2")
)
dimension = 1024 #1536
print("Bedrock Embeddings Model Loaded")

Bedrock Embeddings Model Loaded


## 3. Depoly ReRanker model (if needed)

In [10]:
import json
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
depoly = False

In [12]:
if depoly:

    try:
        role = sagemaker.get_execution_role()
    except ValueError:
        iam = boto3.client('iam')
        role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

    # Hub Model configuration. https://huggingface.co/models
    hub = {
        'HF_MODEL_ID':'Dongjin-kr/ko-reranker',
        'HF_TASK':'text-classification'
    }

    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        transformers_version='4.26.0',
        pytorch_version='1.13.1',
        py_version='py39',
        env=hub,
        role=role, 
    )

    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.g5.xlarge' # instance type
    )

    print(f'Accept: {predictor.accept}')
    print(f'ContentType: {predictor.content_type}')
    print(f'Endpoint: {predictor.endpoint}')

#### Save reranker endpoint to Parameter Store

In [13]:
if depoly:

    import boto3
    from utils.ssm import parameter_store

    region=boto3.Session().region_name
    pm = parameter_store(region)

    pm.put_params(
        key="reranker_endpoint",
        value=f'{predictor.endpoint}',
        overwrite=True,
        enc=False
    )

## 4. Invocation (prediction)

In [14]:
from utils.ssm import parameter_store

In [15]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [16]:
runtime_client = boto3.Session().client('sagemaker-runtime')
print (f'runtime_client: {runtime_client}')

runtime_client: <botocore.client.SageMakerRuntime object at 0x7f79025857b0>


In [17]:
endpoint_name = pm.get_params(
    key="reranker_endpoint",
    enc=False
)
deserializer = "application/json"

In [18]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "I hate you", "text_pair": "I don't like you"},
            {"text": "He hates you", "text_pair": "He like you"}
        ]
    }
)

In [19]:
payload = json.dumps(
    {
        "inputs": [
            {"text": "나는 너를 사랑하지 않아", "text_pair": "나는 너를 좋아하지 않아"},
            {"text": "그는 너를 싫어해", "text_pair": "그는 너를 좋아해"}
        ]
    }
)

In [20]:
%%time
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Accept=deserializer,
    Body=payload
)
## deserialization
out = json.loads(response['Body'].read().decode()) ## for json
print (f'Response: {out}')

Response: [{'label': 'LABEL_0', 'score': 0.9984032511711121}, {'label': 'LABEL_0', 'score': 0.004707992542535067}]
CPU times: user 13.9 ms, sys: 257 µs, total: 14.2 ms
Wall time: 87.3 ms


## 5. LangChain OpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [21]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [22]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력
- parent document 용으로 생성된 index 사용할 것 

In [23]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### OpenSearch Client 생성

In [24]:
from utils.opensearch import opensearch_utils

In [25]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 6. Retriever based on Hybrid Search + ParentDocument + ReRanker
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- Hybrid-Search에 대한 자세한 내용는 **"01_rag_hybrid_search.ipyno"** 에서 확인 가능합니다.
- [Parent Document](https://towardsdatascience.com/forget-rag-the-future-is-rag-fusion-1147298d8ad1)
![parent-document.png](../../../imgs/parent-document.png)

## Parent documents in Hybrid search
- Lexical search: parent documents only
- Semantic search: child documents first, getting parent documents corresponding to that child document

In [26]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [27]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    os_client=os_client,
    index_name=index_name,
    llm_text=llm_text, # llm for query augmentation in both rag_fusion and HyDE
    llm_emb=llm_emb, # Used in semantic search based on opensearch 

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for search
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for semantic, for lexical], Semantic, Lexical search 결과에 대한 최종 반영 비율 정의
    reranker=True, # enable reranker with reranker model
    reranker_endpoint_name=endpoint_name, # endpoint name for reranking model
    parent_document = True, # enable parent document

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [28]:
from utils.rag import show_context_used

In [29]:
#query = "vidio max size?"
query = "What is knox?"

In [30]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.invoke(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

===== ParentDocument =====
filter: [{'bool': {'should': [{'term': {'metadata.family_tree': 'child'}}, {'term': {'metadata.family_tree': 'parent_table'}}, {'term': {'metadata.family_tree': 'parent_image'}}]}}]
# child_docs: 5
# parent docs: 5
# duplicates: 0
##############################
async_mode
##############################
True
##############################
reranker
##############################
True
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
True
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better manage your fleet of devices. It includes individual services such as K

- update parameters
    - **parent_document=False**

In [31]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    #filter=[],
    filter=[
        #{'term': {'metadata.project': 'KS'}},
        #{'term': {'metadata.family_tree': 'child'}},
    ],
    reranker=True,
    reranker_endpoint_name=endpoint_name,
    parent_document=False, # enable parent document
    verbose=True,
)

In [32]:
query = "What is knox?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

/tmp/ipykernel_32381/3505594201.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)


##############################
async_mode
##############################
True
##############################
reranker
##############################
True
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['What is Knox Suite?. Knox Suite is a bundled offering designed to help enterprise IT admins better manage your fleet of devices. It includes individual services such as Knox Platform for Enterprise, Knox Mobile Enrollment, Knox Manage, Knox E-FOTA, and Knox Asset Intelligence. Key features of Knox Suite include: Secure - Ensure your business data is protected with managed security features at your control. Deploy - Enroll c

## 5. RAG using RetrievalQA powered by LangChain

In [33]:
from textwrap import dedent

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [37]:
system_prompt = dedent(
    """
    You are a master answer bot designed to answer user's questions.
    I'm going to give you contexts which consist of texts, tables and images.
    Read the contexts carefully, because I'm going to ask you a question about it.
    """
)

human_prompt = dedent(
    """
    Here is the contexts as texts: <contexts>{contexts}</contexts>

    First, find a few paragraphs or sentences from the contexts that are most relevant to answering the question.
    Then, answer the question as much as you can.

    Skip the preamble and go straight into the answer.
    Don't insert any XML tag such as <contexts> and </contexts> when answering.
    Answer in Korean.

    Here is the question: <question>{question}</question>

    If the question cannot be answered by the contexts, say "No relevant contexts".
    """
)

### Update Search Params (Optional)

In [38]:
from utils.rag import rag_chain
from langchain.schema.output_parser import StrOutputParser

In [39]:
opensearch_hybrid_retriever.update_search_params(
    k=6,
    minimum_should_match=0,
    # filter=[
    #     {'term': {'metadata.family_tree': 'child'}},
    # ],
    ensemble_weights=[0.51, 0.49], #semantic, lexical

    reranker=True,
    reranker_endpoint_name=endpoint_name,

    parent_document=True, # enable parent document
    verbose=False
)

### Request

In [40]:
qa = rag_chain(
    llm_text=llm_text,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    human_prompt=human_prompt,
    return_context=True,
    verbose=False,
    #multi_turn=True
)

In [41]:
#query = "중지된 경우 이체"
query = "vidio max size?"
#query = "How does Knox E-FOTA enable enterprises to deploy OS updates remotely without requiring user interaction?"
#query = "What feature of Knox E-FOTA allows IT administrators to ensure that devices have the same OS version installed across the entire fleet of devices?"
#query = "How does Knox E-FOTA allow IT admins to securely push updates to enterprise mobile devices while maintaining compatibility with in-house apps?"
#query = "What are the three editions of Knox E-FOTA service that have different features?"
#query = "How do you obtain the Knox E-FOTA client APK that is needed to install the app through an EMM?"

response, contexts = qa.invoke(
    query = query
)

#show_context_used(contexts)

관련 컨텍스트:

Previously, the maximum content size was 300 MB. In 20.08, this limit was increased to 1.0 GB.

답변:

이전에는 최대 콘텐츠 크기가 300MB였습니다. 20.08 버전에서 이 제한이 1.0GB로 증가되었습니다. 따라서 현재 비디오 파일의 최대 크기는 1.0GB입니다.

In [42]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response)

print (colored("\n\n### Contexts ###", "green"))
show_context_used(contexts)

##################################
query:  vidio max size?
##################################


### Answer ###
관련 컨텍스트:

Previously, the maximum content size was 300 MB. In 20.08, this limit was increased to 1.0 GB.

답변:

이전에는 최대 콘텐츠 크기가 300MB였습니다. 20.08 버전에서 이 제한이 1.0GB로 증가되었습니다. 따라서 현재 비디오 파일의 최대 크기는 1.0GB입니다.


### Contexts ###

-----------------------------------------------
1. Chunk: 4007 Characters
-----------------------------------------------
. When adding or modifying a Windows profile, you can find this setting in Windows > Interface >
Removable Storage. The USB policy ( Interface > USB ) only applies to Windows mobile devices. ##VPN
update for Windows devices , You can now allow (or disallow) end users to change their VPN settings.
When adding or modifying a Windows profile, you can find this setting in Windows > System > VPN.
When setting up the VPN configuration on devices, you can distribute end users across the following
VPN clients: Pulse Secure , Check Point Capsule V